# Notebook 2: Model Training and Evaluation

This notebook focuses on training a machine learning model using the preprocessed data from Notebook 1. We will:
1. Load the cleaned data.
2. Split the data into training and testing sets.
3. Build a `scikit-learn` pipeline for vectorization and classification.
4. Train the model.
5. Evaluate its performance using key metrics (Precision, Recall, F1-Score) and a confusion matrix.
6. Perform a qualitative error analysis to understand its weaknesses.
7. Save the final trained model for future use.


## 1. Setup and Data Loading


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

sns.set_context('talk')

# Load the processed dataset
try:
    df = pd.read_csv('../data/cleaned_data.csv')
    print("Processed dataset loaded successfully!")
    df.dropna(subset=['processed_text', 'bullying_label'], inplace=True)
except FileNotFoundError:
    print("Error: `data/cleaned_data.csv` not found.")
    print("Please run Notebook 1 or `src/preprocess.py` first.")


## 2. Prepare Data for Modeling


In [ ]:
# Define our features (X) and target (y)
X = df['processed_text']
y = df['bullying_label']

# Split data into training and testing sets
# We use stratify=y to ensure the class distribution is the same in both train and test sets, which is crucial for imbalanced datasets.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")


## 3. Build and Train the Model Pipeline

We will use a `Pipeline` to chain our vectorizer and classifier together. This is a best practice as it prevents data leakage from the test set during the feature engineering phase.


In [ ]:
triage_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1, 2))),
    ('clf', LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000, C=1.0))
])

# Train the pipeline on the training data
print("Training the model...")
triage_pipeline.fit(X_train, y_train)
print("Training complete!")


**Pipeline Components:**
- **TfidfVectorizer**: Converts text into numerical features, considering both unigrams and bigrams (`ngram_range=(1, 2)`) to capture short phrases.
- **LogisticRegression**: A robust and interpretable linear model. We use `class_weight='balanced'` to counteract class imbalance.


## 4. Evaluate Model Performance


In [ ]:
# Make predictions on the test set
y_pred = triage_pipeline.predict(X_test)

# Print a detailed classification report
print("--- Classification Report ---")
report = classification_report(y_test, y_pred, target_names=['Not Bullying', 'Bullying'])
print(report)


The F1-score for the **Bullying** class is our key metric. It provides a balance between Precision (minimizing false positives) and Recall (minimizing false negatives).


In [ ]:
# Generate and display the confusion matrix
print("--- Confusion Matrix ---")
cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Not Bullying', 'Bullying'])

fig, ax = plt.subplots(figsize=(8, 6))
disp.plot(ax=ax, cmap='Blues')
plt.title('Confusion Matrix')
plt.show()


**Interpreting the Matrix:**
- **Top-Left (True Negative):** Correctly identified as 'Not Bullying'.
- **Bottom-Right (True Positive):** Correctly identified as 'Bullying'.
- **Top-Right (False Positive):** Mistakenly flagged as 'Bullying'. **(We want this to be low)**
- **Bottom-Left (False Negative):** Missed a case of 'Bullying'. **(We also want this to be low)**


## 5. Qualitative Error Analysis

Metrics are useful, but looking at the actual errors our model makes provides deeper insight into its limitations.


In [ ]:
# Create a DataFrame to see the errors
error_df = pd.DataFrame({'text': X_test, 'actual': y_test, 'predicted': y_pred})
error_df = error_df[error_df['actual'] != error_df['predicted']]

# Show some False Positives (predicted Bullying, but was Not Bullying)
print("--- Sample False Positives ---")
fp_samples = error_df[error_df['actual'] == 0].head(5)
for i, row in fp_samples.iterrows():
    print(f"Text: {row['text']}\n")

# Show some False Negatives (predicted Not Bullying, but was Bullying)
print("\n--- Sample False Negatives ---")
fn_samples = error_df[error_df['actual'] == 1].head(5)
for i, row in fn_samples.iterrows():
    print(f"Text: {row['text']}\n")


**Analysis of Errors:**
The **False Positives** often involve aggressive language or profanity used in a non-bullying context (e.g., excitement, friendly banter). This is a classic challenge for NLP models.

The **False Negatives** are more concerning. They often involve subtle sarcasm, exclusion, or targeted attacks that don't use obvious keywords. This is precisely where a more advanced, context-aware model like DistilBERT or an LLM would be needed.


## 6. Save the Final Model


In [ ]:
# Create a directory for the models if it doesn't exist
model_dir = '../models'
os.makedirs(model_dir, exist_ok=True)

# Save the entire pipeline object
model_path = os.path.join(model_dir, 'triage_model_pipeline.joblib')
joblib.dump(triage_pipeline, model_path)

print(f"Model pipeline saved successfully to: {model_path}")


## 7. Conclusion

We have successfully trained and evaluated a Logistic Regression model that serves as a strong baseline for cyberbullying detection. Its performance is respectable, and its weaknesses (difficulty with nuance and sarcasm) clearly point toward the next steps outlined in the project report: implementing a more advanced deep learning model to handle the ambiguous cases this Triage Model struggles with.
